# 版图电容预测

## 基本库导入

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append(os.path.join(os.getcwd(), '..'))

from config import *
from data.layout import convert_data, convert_data_parallel

## 数据导入与数据清洗

In [2]:
def data_process(x, reserve_num):
	raw_num = np.array([len(i) for i in x]).reshape(-1, 1)
	valid_num = np.array([reserve_num if len(i) > reserve_num else len(i) for i in x]).reshape(-1, 1)
	new_x = np.zeros(shape=(len(x), reserve_num, 5), dtype=np.float32)
	for i in range(len(x)):
		num = min(len(x[i]), reserve_num)
		new_x[i][:num] = x[i][:num]

	return new_x, valid_num, raw_num

def data_process2(x, y, thresh): # [400 2500]
	thresh = np.array(thresh, dtype=np.int32)
	raw_num = np.array([len(i) for i in x]).reshape(-1, 1)
	valid_num = np.array([thresh[1] if len(i) > thresh[1] else len(i) for i in x], dtype=np.int32).reshape(-1, 1)
	sum_reserve = np.sum(valid_num >= thresh[0], dtype=np.int32)
	new_x = np.zeros(shape=(sum_reserve, thresh[1], 5), dtype=np.float32)
	new_y = np.zeros(shape=(sum_reserve, 1), dtype=np.float32)
	index = 0
	for i, num in enumerate(valid_num):
		num = num[0]
		if num >= thresh[0]:
			new_x[index][:num] = x[i][:num]
			new_y[index][0] = y[i][0]
			index += 1
	valid_num = valid_num[valid_num>=thresh[0]].reshape(-1, 1)
	new_y = np.concatenate([valid_num, new_y], axis=1)

	return new_x, new_y, raw_num

x_total = []
x_couple = []
y_total = []
y_couple = []
raw_nums = []
if len(pattern_nums) == 0:
	print("pattern_nums is empty, please check the config.py")
for pattern_num in pattern_nums:
	dir_prj = "D:/learn_more_from_life/computer/EDA/work/prj/rc_predict/"
	dir_load = os.path.join(dir_prj, "data/convert_data/pattern{}".format(pattern_num))
	if not os.path.exists(dir_load):
		# convert_data(pattern_num)
		convert_data_parallel(pattern_num, num_process=8)

# data process 1
# x_total_ = np.load(os.path.join(dir_load, "x_total.npy"), allow_pickle=True)
# x_total_, total_valid_num, total_raw_nums = data_process(x_total_, reserve_num)
# x_couple_ = np.load(os.path.join(dir_load, "x_couple.npy"), allow_pickle=True)
# x_couple_, couple_valid_num, _ = data_process(x_couple_, reserve_num)
# y_total_ = np.load(os.path.join(dir_load, "y_total.npy")).reshape(-1, 1)
# y_total_ = np.concatenate([total_valid_num, y_total_], axis=1)
# y_couple_ = np.load(os.path.join(dir_load, "y_couple.npy")).reshape(-1, 1)
# y_couple_ = np.concatenate([couple_valid_num, y_couple_], axis=1)

# data process 2
x_total_ = np.load(os.path.join(dir_load, "x_total.npy"), allow_pickle=True)
y_total_ = np.load(os.path.join(dir_load, "y_total.npy")).reshape(-1, 1)
x_total_, y_total_, total_raw_nums = data_process2(x_total_, y_total_, thresh)
x_couple_ = np.load(os.path.join(dir_load, "x_couple.npy"), allow_pickle=True)
y_couple_ = np.load(os.path.join(dir_load, "y_couple.npy")).reshape(-1, 1)
x_couple_, y_couple_, _ = data_process2(x_couple_, y_couple_, thresh)

# concatenate data
if len(x_total) == 0:
	x_total = x_total_.copy()
	x_couple = x_couple_.copy()
	y_total = y_total_.copy()
	y_couple = y_couple_.copy()
	raw_nums = total_raw_nums.copy()
else:
	x_total = np.concatenate([x_total, x_total_], axis=0)
	x_couple = np.concatenate([x_couple, x_couple_], axis=0)
	y_total = np.concatenate([y_total, y_total_], axis=0)
	y_couple = np.concatenate([y_couple, y_couple_], axis=0)
	raw_nums = np.concatenate([raw_nums, total_raw_nums],  axis=0)
print("load data from {}".format(dir_load))

print('raw nums shape:', raw_nums.shape)
print('raw nums mean:', np.mean(raw_nums))
print('raw nums max:', np.max(raw_nums))
print('raw nums min:', np.min(raw_nums))

valid_num = y_total[:, 0]
print('valid nums shape:', valid_num.shape)
print('valid nums mean:', np.mean(valid_num))
print('valid nums max:', np.max(valid_num))
print('valid nums min:', np.min(valid_num))

print(x_total.shape)
print('x total first 10 samples:')
print(x_total[0][:10])
print('x couple first 10 samples:')
print(x_couple[0][:10])

print(y_total.shape)
print('y total first 10 samples:')
print(y_total[:10])
print('y couple first 10 samples:')
print(y_couple[:10])

load data from D:/learn_more_from_life/computer/EDA/work/prj/rc_predict/data/convert_data/pattern1
raw nums shape: (1456, 1)
raw nums mean: 1891.4285714285713
raw nums max: 12501
raw nums min: 149
valid nums shape: (1182,)
valid nums mean: 1591.3883248730965
valid nums max: 2500.0
valid nums min: 405.0
(1182, 2500, 5)
x total first 10 samples:
[[ 0.00e+00  0.00e+00  0.00e+00  2.40e-02  3.00e+01]
 [ 1.52e-01  0.00e+00  0.00e+00  2.40e-02  3.00e+01]
 [-1.52e-01  0.00e+00  0.00e+00  2.40e-02  3.00e+01]
 [ 3.04e-01  0.00e+00  0.00e+00  2.40e-02  3.00e+01]
 [-3.04e-01  0.00e+00  0.00e+00  2.40e-02  3.00e+01]
 [ 4.56e-01  0.00e+00  0.00e+00  2.40e-02  3.00e+01]
 [-4.56e-01  0.00e+00  0.00e+00  2.40e-02  3.00e+01]
 [ 6.08e-01  0.00e+00  0.00e+00  2.40e-02  3.00e+01]
 [-6.08e-01  0.00e+00  0.00e+00  2.40e-02  3.00e+01]
 [ 7.60e-01  0.00e+00  0.00e+00  2.40e-02  3.00e+01]]
x couple first 10 samples:
[[ 0.00000e+00  0.00000e+00  0.00000e+00  2.40000e-02  3.00000e+01]
 [ 0.00000e+00 -1.51280e+01 

## 数据分割

In [3]:
# data split 6:2:2
from sklearn.model_selection import train_test_split


x_total_train, x_total_valid_test, y_total_train, y_total_valid_test = train_test_split(x_total, y_total, test_size=0.4, random_state=seed, shuffle=True)
x_total_valid, x_total_test, y_total_valid, y_total_test = train_test_split(x_total_valid_test, y_total_valid_test, test_size=0.5, random_state=seed, shuffle=True)
print('total x len:', len(x_total_train), len(x_total_valid), len(x_total_test))
print('x total train first 10\n', x_total_train[0][:10])
print('x total train last 10\n', x_total_train[0][-10:])
print('y total train first 10\n', y_total_train[:10])

x_couple_train, x_couple_valid_test, y_couple_train, y_couple_valid_test = train_test_split(x_couple, y_couple, test_size=0.4, random_state=seed, shuffle=True)
x_couple_valid, x_couple_test, y_couple_valid, y_couple_test = train_test_split(x_couple_valid_test, y_couple_valid_test, test_size=0.5, random_state=seed, shuffle=True)
print('couple x len', len(x_couple_train), len(x_couple_valid), len(x_couple_test))
print('x couple train first 10\n', x_couple_train[0][:10])
print('x couple train last 10\n', x_couple_train[0][-10:])
print('y couple train first 10\n', y_couple_train[:10])

total x len: 709 236 237
x total train first 10
 [[ 0.     0.     0.     0.064 30.   ]
 [ 0.184  0.     0.     0.064 30.   ]
 [-0.184  0.     0.     0.064 30.   ]
 [-0.368  0.     0.     0.064 30.   ]
 [ 0.368  0.     0.     0.064 30.   ]
 [ 0.552  0.     0.     0.064 30.   ]
 [-0.552  0.     0.     0.064 30.   ]
 [ 0.736  0.     0.     0.064 30.   ]
 [-0.736  0.     0.     0.064 30.   ]
 [ 0.92   0.     0.     0.064 30.   ]]
x total train last 10
 [[-27.968 -45.24    1.      0.064  60.   ]
 [ 28.152 -45.24    1.      0.064  60.   ]
 [-28.152 -45.24    1.      0.064  60.   ]
 [ 51.152  15.      1.      0.064  60.   ]
 [-51.152  15.      1.      0.064  60.   ]
 [ 28.336 -45.24    1.      0.064  60.   ]
 [-28.336 -45.24    1.      0.064  60.   ]
 [ 28.52  -45.24    1.      0.064  60.   ]
 [-28.52  -45.24    1.      0.064  60.   ]
 [ 51.336  15.      1.      0.064  60.   ]]
y total train first 10
 [[2500.            3.69232702]
 [2029.            6.56918097]
 [2189.            4.42918301]

## 数据标准化

In [4]:
from sklearn.preprocessing import StandardScaler


def cal_mean_std(x, y):
    num_s = 0
    num_sum = round(np.sum(y[:, 0]))
    x_valid = np.zeros((num_sum, 5), dtype=np.float32)
    for i in range(len(x)):
        num = round(y[i][0])
        x_valid[num_s:num_s+num, :] = x[i][:num]
        num_s += num
    mean = np.mean(x_valid, axis=0)
    std = np.std(x_valid, axis=0)
    std[std == 0] = 1

    return mean, std

def get_mask(x, y):
    mask = np.zeros(shape=x.shape, dtype=np.int32)
    for i in range(len(x)):
        num = round(y[i][0])
        mask[i][:num] = 1

    return mask

def data_process(x, y, mean, std):
    mask = get_mask(x, y)

    return np.multiply((x - mean) / std, mask)

# x
# total
mean_total, std_total = cal_mean_std(x_total_train, y_total_train)
print('mean total:', mean_total)
print('std total:', std_total)
x_total_train_std = data_process(x_total_train, y_total_train, mean_total, std_total)
x_total_train_std_flat = x_total_train_std.reshape(len(x_total_train_std), -1)
x_total_valid_std = data_process(x_total_valid, y_total_valid, mean_total, std_total)
x_total_valid_std_flat = x_total_valid_std.reshape(len(x_total_valid_std), -1)
x_total_test_std = data_process(x_total_test, y_total_test, mean_total, std_total)
x_total_test_std_flat = x_total_test_std.reshape(len(x_total_test_std), -1)
print('x total train std flat first 20\n', x_total_train_std_flat[0][:20])
print('x total train std flat last 20\n', x_total_train_std_flat[0][-20:])

# couple
mean_couple, std_couple = cal_mean_std(x_couple_train, y_couple_train)
print('mean couple:', mean_couple)
print('std couple:', std_couple)
x_couple_train_std = data_process(x_couple_train, y_couple_train, mean_couple, std_couple)
x_couple_train_std_flat = x_couple_train_std.reshape(len(x_couple_train_std), -1)
x_couple_valid_std = data_process(x_couple_valid, y_couple_valid, mean_couple, std_couple)
x_couple_valid_std_flat = x_couple_valid_std.reshape(len(x_couple_valid_std), -1)
x_couple_test_std = data_process(x_couple_test, y_couple_test, mean_couple, std_couple)
x_couple_test_std_flat = x_couple_test_std.reshape(len(x_couple_test_std), -1)
print('x couple train std flat first 20\n', x_couple_train_std_flat[0][:20])
print('x couple train std flat last 20\n', x_couple_train_std_flat[0][-20:])

# y
# total
scaler_yt = StandardScaler()
y_total_train_std = y_total_train[:, 1].copy().reshape(-1, 1)
# scaler_yt.fit(y_total_train_std)
# y_total_train_std = scaler_yt.transform(y_total_train_std)
# print('y total train std first 10\n', y_total_train_std[:10].reshape(-1))

# # couple
scaler_yc = StandardScaler()
y_couple_train_std = y_couple_train[:, 1].copy().reshape(-1, 1)
# scaler_yc.fit(y_couple_train_std)
# y_couple_train_std = scaler_yc.transform(y_couple_train_std)
# print('y couple train std first 10\n', y_couple_train_std[:10].reshape(-1))

if DISABLE_STD:
    x_total_train_std_flat = x_total_train.reshape(len(x_total_train), -1)
    x_total_valid_std_flat = x_total_valid.reshape(len(x_total_valid), -1)
    x_total_test_std_flat = x_total_test.reshape(len(x_total_test), -1)
    x_couple_train_std_flat = x_couple_train.reshape(len(x_couple_train), -1)
    x_couple_valid_std_flat = x_couple_valid.reshape(len(x_couple_valid), -1)
    x_couple_test_std_flat = x_couple_test.reshape(len(x_couple_test), -1)


mean total: [ 5.4391143e-03 -1.3546277e+01 -8.4551439e-02  2.1682155e-01
  4.6763111e+01]
std total: [40.622883  27.673521   0.5854825  3.6689255 15.044824 ]
x total train std flat first 20
 [-1.33892871e-04  4.89503205e-01  1.44413263e-01 -4.16529402e-02
 -1.11421120e+00  4.39557387e-03  4.89503205e-01  1.44413263e-01
 -4.16529402e-02 -1.11421120e+00 -4.66335984e-03  4.89503205e-01
  1.44413263e-01 -4.16529402e-02 -1.11421120e+00 -9.19282623e-03
  4.89503205e-01  1.44413263e-01 -4.16529402e-02 -1.11421120e+00]
x total train std flat last 20
 [-0.69767183 -1.14527261  1.85240638 -0.04165294  0.87983012  0.70193344
 -1.14527261  1.85240638 -0.04165294  0.87983012 -0.70220131 -1.14527261
  1.85240638 -0.04165294  0.87983012  1.26358736  1.03153753  1.85240638
 -0.04165294  0.87983012]
mean couple: [ 3.8759136e-03 -1.3543772e+01 -8.4553182e-02  1.0264515e-01
  4.6673248e+01]
std couple: [40.62001    27.676252    0.58555335  3.741425   15.194086  ]
x couple train std flat first 20
 [-9.541

## 命名简化

In [5]:
# total 
xt_train = x_total_train_std_flat
xt_valid = x_total_valid_std_flat
xt_test = x_total_test_std_flat
yt_train = y_total_train[:, 1].reshape(-1, 1)
yt_valid = y_total_valid[:, 1].reshape(-1, 1)
yt_test = y_total_test[:, 1].reshape(-1, 1)

# couple
xc_train = x_couple_train_std_flat
xc_valid = x_couple_valid_std_flat
xc_test = x_couple_test_std_flat
yc_train = y_couple_train[:, 1].reshape(-1, 1)
yc_valid = y_couple_valid[:, 1].reshape(-1, 1)
yc_test = y_couple_test[:, 1].reshape(-1, 1)

## 模型预测

#### 模型评估

In [6]:
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error


def model_analysis(model, scaler, x_train, y_train, x_valid, y_valid, x_test, y_test, name):
    model.fit(x_train, y_train.ravel())
    y_valid_predict = model.predict(x_valid).reshape(-1, 1)
    y_test_predict = model.predict(x_test).reshape(-1, 1)
    mse_train = mean_squared_error(y_train, model.predict(x_train))
    mse_valid = mean_squared_error(y_valid, y_valid_predict)
    mse_test = mean_squared_error(y_test, y_test_predict)
    r2_train = r2_score(y_train, model.predict(x_train))
    r2_valid = r2_score(y_valid, y_valid_predict)
    r2_test = r2_score(y_test, y_test_predict)
    print(f"{name} train mse:", mse_train)
    print(f"{name} train r2:", r2_train)
    print(f"{name} valid mse:", mse_valid)
    print(f"{name} valid r2:", r2_valid)
    print(f"{name} test mse:", mse_test)
    print(f"{name} test r2:", r2_test)
    print('')
    analysis_result(y_valid, y_valid_predict, title=f'{name} valid analysis')
    analysis_result(y_test, y_test_predict, title=f'{name} test analysis')
    print('')


def analysis_result(y, y_predict, title):
    relative_error = np.abs(y - y_predict) / y
    max_error = np.max(relative_error)
    mean_error = np.mean(relative_error)
    std_error = np.std(relative_error)
    num_good = np.sum(relative_error <= 0.05) / len(y)
    num_bad = np.sum(relative_error > 0.05) / len(y)
    print(title)
    print('max error:', max_error)
    print('mean error:', mean_error)
    print('std error:', std_error)
    print('good ratio:', num_good)
    print('bad ratio:', num_bad)
    print('')


### 线性回归

In [7]:
from sklearn.linear_model import LinearRegression


# total
lr_t = LinearRegression()
model_analysis(lr_t, scaler_yt, xt_train, yt_train, xt_valid, yt_valid, xt_test, yt_test, 'linear total')

# couple
lr_c = LinearRegression()
model_analysis(lr_c, scaler_yc, xc_train, yc_train, xc_valid, yc_valid, xc_test, yc_test, 'linear couple')

linear total train mse: 8.47674850050666e-05
linear total train r2: 0.999999999689405
linear total valid mse: 536592.8243586249
linear total valid r2: -1.0383904869948921
linear total test mse: 504243.10885324265
linear total test r2: -0.7415393763308538

linear total valid analysis
max error: 241.70183219871927
mean error: 61.166491484343986
std error: 58.008205626060274
good ratio: 0.00847457627118644
bad ratio: 0.9915254237288136

linear total test analysis
max error: 255.90134225719584
mean error: 60.42680824499129
std error: 55.44910051923187
good ratio: 0.008438818565400843
bad ratio: 0.9915611814345991


linear couple train mse: 8.976672376705751e-10
linear couple train r2: 0.9999999999451429
linear couple valid mse: 25.253270032469192
linear couple valid r2: -0.949905708302961
linear couple test mse: 29.29431902654853
linear couple test r2: -0.8640840420161036

linear couple valid analysis
max error: 3598.108836511257
mean error: 97.50434161396984
std error: 384.9840285299149
g

### 支持向量机回归

In [8]:
from sklearn.svm import SVR

# total
# linear svr
lr_svf_t = SVR(kernel='linear')
model_analysis(lr_svf_t, scaler_yt, xt_train, yt_train, xt_valid, yt_valid, xt_test, yt_test, 'linear svr total')
# poly svr
poly_svf_t = SVR(kernel='poly')
model_analysis(poly_svf_t, scaler_yt, xt_train, yt_train, xt_valid, yt_valid, xt_test, yt_test, 'poly svr total')
# rbf svr
rbf_svf_t = SVR(kernel='rbf')
model_analysis(rbf_svf_t, scaler_yt, xt_train, yt_train, xt_valid, yt_valid, xt_test, yt_test, 'rbf svr total')

# couple
# linear svr
lr_svf_c = SVR(kernel='linear')
model_analysis(lr_svf_c, scaler_yc, xc_train, yc_train, xc_valid, yc_valid, xc_test, yc_test, 'linear svr couple')
# poly svr
poly_svf_c = SVR(kernel='poly')
model_analysis(poly_svf_c, scaler_yc, xc_train, yc_train, xc_valid, yc_valid, xc_test, yc_test, 'poly svr couple')
# rbf svr
rbf_svf_c = SVR(kernel='rbf')
model_analysis(rbf_svf_c, scaler_yc, xc_train, yc_train, xc_valid, yc_valid, xc_test, yc_test, 'rbf svr couple')

linear svr total train mse: 0.010074185835932437
linear svr total train r2: 0.999999963087366
linear svr total valid mse: 537550.2013073325
linear svr total valid r2: -1.0420273378362097
linear svr total test mse: 502576.3588989684
linear svr total test r2: -0.7357828064840835

linear svr total valid analysis
max error: 241.28670247346258
mean error: 61.140865088239266
std error: 57.94046333849526
good ratio: 0.00847457627118644
bad ratio: 0.9915254237288136

linear svr total test analysis
max error: 255.19500909754444
mean error: 60.26372554908615
std error: 55.11207950595409
good ratio: 0.008438818565400843
bad ratio: 0.9915611814345991


poly svr total train mse: 298317.82593615114
poly svr total train r2: -0.09306070761704732
poly svr total valid mse: 287592.47512185323
poly svr total valid r2: -0.09249646810018519
poly svr total test mse: 319212.72011684376
poly svr total test r2: -0.10248709749043217

poly svr total valid analysis
max error: 1.9990150325954703
mean error: 0.45606

### K近邻回归

In [9]:
from sklearn.neighbors import KNeighborsRegressor

# total 
# uniform knn
uni_knn_t = KNeighborsRegressor(n_neighbors=5, weights='uniform')
model_analysis(uni_knn_t, scaler_yt, xt_train, yt_train, xt_valid, yt_valid, xt_test, yt_test, 'uniform knn total')
# distance knn
dis_knn_t = KNeighborsRegressor(n_neighbors=5, weights='distance')
model_analysis(dis_knn_t, scaler_yt, xt_train, yt_train, xt_valid, yt_valid, xt_test, yt_test, 'distance knn total')

# couple
# uniform knn
uni_knn_c = KNeighborsRegressor(n_neighbors=5, weights='uniform')
model_analysis(uni_knn_c, scaler_yc, xc_train, yc_train, xc_valid, yc_valid, xc_test, yc_test, 'uniform knn couple')
# distance knn
dis_knn_c = KNeighborsRegressor(n_neighbors=5, weights='distance')
model_analysis(dis_knn_c, scaler_yc, xc_train, yc_train, xc_valid, yc_valid, xc_test, yc_test, 'distance knn couple')

uniform knn total train mse: 188170.68607699528
uniform knn total train r2: 0.3105273456902714
uniform knn total valid mse: 280425.60640241514
uniform knn total valid r2: -0.065271212084685
uniform knn total test mse: 268991.544827428
uniform knn total test r2: 0.07096525665485198

uniform knn total valid analysis
max error: 196.61868273180883
mean error: 12.991087618486379
std error: 38.25645967317624
good ratio: 0.09322033898305085
bad ratio: 0.9067796610169492

uniform knn total test analysis
max error: 178.04224991767614
mean error: 16.37395494137178
std error: 37.559734397617945
good ratio: 0.09282700421940929
bad ratio: 0.9071729957805907


distance knn total train mse: 1.0201123890001656e-08
distance knn total train r2: 0.9999999999999626
distance knn total valid mse: 286109.5744256274
distance knn total valid r2: -0.0868632756027683
distance knn total test mse: 270844.9324306402
distance knn total test r2: 0.06456408342327691

distance knn total valid analysis
max error: 196.60

### 回归树

In [10]:
from sklearn.tree import DecisionTreeRegressor

# total
dtr_t = DecisionTreeRegressor()
model_analysis(dtr_t, scaler_yt, xt_train, yt_train, xt_valid, yt_valid, xt_test, yt_test, 'decision tree total')

# couple
dtr_c = DecisionTreeRegressor()
model_analysis(dtr_c, scaler_yc, xc_train, yc_train, xc_valid, yc_valid, xc_test, yc_test, 'decision tree couple')

decision tree total train mse: 0.0
decision tree total train r2: 1.0
decision tree total valid mse: 107013.66556436823
decision tree total valid r2: 0.59348014367208
decision tree total test mse: 178155.68601359206
decision tree total test r2: 0.38469135846890157

decision tree total valid analysis
max error: 159.37860399315338
mean error: 2.221263431969645
std error: 16.23321874985279
good ratio: 0.7415254237288136
bad ratio: 0.2584745762711864

decision tree total test analysis
max error: 172.13935220989532
mean error: 5.015313934213524
std error: 23.944102476704234
good ratio: 0.6582278481012658
bad ratio: 0.34177215189873417


decision tree couple train mse: 0.0
decision tree couple train r2: 1.0
decision tree couple valid mse: 0.36866501617504627
decision tree couple valid r2: 0.9715339035868601
decision tree couple test mse: 0.661504788728646
decision tree couple test r2: 0.9579064965029983

decision tree couple valid analysis
max error: 148.9297913558647
mean error: 0.7603211676

### 集成模型

#### 基础模型

In [11]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor


# total
# random forest
rfr_t = RandomForestRegressor()
model_analysis(rfr_t, scaler_yt, xt_train, yt_train, xt_valid, yt_valid, xt_test, yt_test, 'random forest total')
# extra tree
etr_t = ExtraTreesRegressor()
model_analysis(etr_t, scaler_yt, xt_train, yt_train, xt_valid, yt_valid, xt_test, yt_test, 'extra tree total')
# gradient boosting
gbr_t = GradientBoostingRegressor()
model_analysis(gbr_t, scaler_yt, xt_train, yt_train, xt_valid, yt_valid, xt_test, yt_test, 'gradient boosting total')


# couple
# random forest
rfr_c = RandomForestRegressor()
model_analysis(rfr_c, scaler_yc, xc_train, yc_train, xc_valid, yc_valid, xc_test, yc_test, 'random forest couple')
# extra tree
etr_c = ExtraTreesRegressor()
model_analysis(etr_c, scaler_yc, xc_train, yc_train, xc_valid, yc_valid, xc_test, yc_test, 'extra tree couple')
# gradient boosting
gbr_c = GradientBoostingRegressor()
model_analysis(gbr_c, scaler_yc, xc_train, yc_train, xc_valid, yc_valid, xc_test, yc_test, 'gradient boosting couple')
# XGBoost extreme gradient boosting

random forest total train mse: 9330.931113366374
random forest total train r2: 0.9658107116680144
random forest total valid mse: 34122.79950986677
random forest total valid r2: 0.8703754751217847
random forest total test mse: 64128.24597997289
random forest total test r2: 0.7785158318511438

random forest total valid analysis
max error: 41.855056441754286
mean error: 3.0565041297101843
std error: 6.47986284638635
good ratio: 0.4194915254237288
bad ratio: 0.5805084745762712

random forest total test analysis
max error: 58.52673220216221
mean error: 4.045175379666759
std error: 8.802115960293436
good ratio: 0.3628691983122363
bad ratio: 0.6371308016877637


extra tree total train mse: 0.0
extra tree total train r2: 1.0
extra tree total valid mse: 33459.703087293376
extra tree total valid r2: 0.8728944231553324
extra tree total test mse: 67540.3169639395
extra tree total test r2: 0.7667313257883286

extra tree total valid analysis
max error: 42.96709567860681
mean error: 1.322397969636417

#### XGBoost

In [12]:
import xgboost as xgb


# total
# XGBoost extreme gradient boosting
xgb_t = xgb.XGBRegressor(booster='gbtree',
                         n_estimators=100,
                         learning_rate=0.1,
                         max_depth=6,
                         min_child_weight=3,
                         seed=42)
model_analysis(xgb_t, scaler_yt, xt_train, yt_train, xt_valid, yt_valid, xt_test, yt_test, 'XGBoost total')

# couple
# XGBoost extreme gradient boosting
xgb_c = xgb.XGBRegressor(booster='gbtree',
                         n_estimators=100,
                         learning_rate=0.1,
                         max_depth=6,
                         min_child_weight=3,
                         seed=42)
model_analysis(xgb_c, scaler_yc, xc_train, yc_train, xc_valid, yc_valid, xc_test, yc_test, 'XGBoost couple')

XGBoost total train mse: 56.46575628857193
XGBoost total train r2: 0.9997931048896216
XGBoost total valid mse: 45799.74754027934
XGBoost total valid r2: 0.8260174839190942
XGBoost total test mse: 72612.47415343834
XGBoost total test r2: 0.749213264929667

XGBoost total valid analysis
max error: 62.09747544879264
mean error: 1.5194832600120618
std error: 5.979551412459218
good ratio: 0.24152542372881355
bad ratio: 0.7584745762711864

XGBoost total test analysis
max error: 77.86104007990843
mean error: 2.374712742249036
std error: 8.399056651045225
good ratio: 0.189873417721519
bad ratio: 0.810126582278481


XGBoost couple train mse: 0.0011668410470530137
XGBoost couple train r2: 0.9999286935331905
XGBoost couple valid mse: 0.12330790036877695
XGBoost couple valid r2: 0.9904789051675768
XGBoost couple test mse: 0.2620276046154352
XGBoost couple test r2: 0.9833264096056072

XGBoost couple valid analysis
max error: 52.372449957228945
mean error: 1.4114008079626859
std error: 5.568089185785

### 懒模型

In [13]:
from lazypredict.Supervised import LazyRegressor
import warnings

warnings.filterwarnings("ignore")

# total
reg_t = LazyRegressor(verbose=0, predictions=True, ignore_warnings=True)
models_t, predictions_t = reg_t.fit(xt_train, xt_valid, yt_train.ravel(), yt_valid.ravel())
print('total models')
print(models_t)
print('')

# couple
reg_c = LazyRegressor(verbose=0, predictions=True, ignore_warnings=True)
models_c, predictions_c = reg_c.fit(xc_train, xc_valid, yc_train.ravel(), yc_valid.ravel())
print('couple models')
print(models_t)

 98%|█████████▊| 41/42 [46:23<00:17, 17.97s/it]   

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.206681 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 640263
[LightGBM] [Info] Number of data points in the train set: 709, number of used features: 12263
[LightGBM] [Info] Start training from score 173.228125
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

100%|██████████| 42/42 [46:35<00:00, 66.56s/it]


total models
                                     Adjusted R-Squared  \
Model                                                     
PoissonRegressor              5953925777742706507776.00   
SGDRegressor                     4957619070821224448.00   
GammaRegressor                                461288.51   
OrthogonalMatchingPursuit                          2.10   
LinearRegression                                   1.29   
TransformedTargetRegressor                         1.29   
KernelRidge                                        1.29   
Ridge                                              1.29   
RidgeCV                                            1.27   
PassiveAggressiveRegressor                         1.27   
LinearSVR                                          1.15   
HuberRegressor                                     1.12   
ElasticNet                                         1.11   
TweedieRegressor                                   1.09   
Lasso                                      

 98%|█████████▊| 41/42 [34:32<00:18, 18.77s/it]   

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.234755 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 643616
[LightGBM] [Info] Number of data points in the train set: 709, number of used features: 12262
[LightGBM] [Info] Start training from score 3.271238
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

100%|██████████| 42/42 [34:43<00:00, 49.61s/it]

couple models
                                     Adjusted R-Squared  \
Model                                                     
PoissonRegressor              5953925777742706507776.00   
SGDRegressor                     4957619070821224448.00   
GammaRegressor                                461288.51   
OrthogonalMatchingPursuit                          2.10   
LinearRegression                                   1.29   
TransformedTargetRegressor                         1.29   
KernelRidge                                        1.29   
Ridge                                              1.29   
RidgeCV                                            1.27   
PassiveAggressiveRegressor                         1.27   
LinearSVR                                          1.15   
HuberRegressor                                     1.12   
ElasticNet                                         1.11   
TweedieRegressor                                   1.09   
Lasso                                     